# Feature Engineering: Stationarity Testing

This notebook tests the stationarity of price and production time series data using the Augmented Dickey-Fuller (ADF) test. The pipeline is designed to be modular and reusable for adding new variables from additional CSV files.

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
import warnings

## 2. Load and Filter Time Series Data

In [ ]:
# Define date range: January 1, 1990 to June 2025
start_date = datetime(1990, 1, 1)
end_date = datetime(2025, 6, 30)

# Define the CSV files and their corresponding column names
# Format: {'csv_filename': 'column_name_in_csv'}
csv_config = {
    'data/beefprice.csv': 'PBEEFUSDM',
    'data/cornprice.csv': 'PMAIZMTUSDM',
    'data/soybeanprice.csv': 'PSMEAUSDM',
    'data/brentoilprice.csv': 'POILBREUSDM',
    'data/lambprice.csv': 'PLAMBUSDM',
    'data/porkprice.csv': 'PPORKUSDM',
    'data/poultryprice.csv': 'PPOULTUSDM',
    'data/bioethanolprod.csv': 'Bioethanol production',  # Special handling for this file
}

# Load and filter data
def load_and_filter_data(csv_path, date_col='observation_date', value_col='PRICE'):
    """
    Load CSV file and filter to date range [1990-01-01, 2025-06-30]
    """
    df = pd.read_csv(csv_path)
    
    # Parse date column
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Filter by date range
    df_filtered = df[(df[date_col] >= start_date) & (df[date_col] <= end_date)].copy()
    
    # Sort by date
    df_filtered = df_filtered.sort_values(date_col).reset_index(drop=True)
    
    # Extract the value column
    return df_filtered, df_filtered[value_col].values

# Test loading one file to check structure
sample_df, _ = load_and_filter_data('data/beefprice.csv', value_col='PBEEFUSDM')
print("Sample data structure:")
print(sample_df.head())
print(f"\nDate range: {sample_df['observation_date'].min()} to {sample_df['observation_date'].max()}")
print(f"Number of observations: {len(sample_df)}")

## 3. Extract Variables and Store in Dictionary

In [ ]:
# Extract all variables and store in a dictionary
# IMPORTANT: Consolidate to 1 observation per MONTH-YEAR (ignore day of month)
variables = {}

for csv_path, value_col in csv_config.items():
    try:
        # Special handling for bioethanol file with semicolon separator
        if 'bioethanol' in csv_path.lower():
            df = pd.read_csv(csv_path, sep=';')
            df['Date'] = pd.to_datetime(df['Date'])
            df_filtered = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)].copy()
            df_filtered = df_filtered.sort_values('Date').reset_index(drop=True)
        else:
            df_filtered, _ = load_and_filter_data(csv_path, value_col=value_col)
        
        # Group by month-year and take the last value of each month
        # This ensures 1 observation per month regardless of the day of month
        if 'bioethanol' in csv_path.lower():
            date_col = 'Date'
            value_col_name = 'Bioethanol production'
        else:
            date_col = 'observation_date'
            value_col_name = value_col
        
        df_filtered['year_month'] = df_filtered[date_col].dt.to_period('M')
        df_monthly = df_filtered.groupby('year_month')[value_col_name].last().reset_index()
        df_monthly['date'] = df_monthly['year_month'].dt.to_timestamp()
        
        values = df_monthly[value_col_name].values
        dates = df_monthly['date'].values
        
        # Use the CSV filename (without .csv) as the variable name
        var_name = csv_path.split('/')[-1].replace('.csv', '')
        
        variables[var_name] = {
            'data': values,
            'dates': dates,
            'length': len(values),
            'csv_path': csv_path
        }
        
        print(f"✓ Loaded {var_name}: {len(values)} observations (1 per month-year)")
    except Exception as e:
        print(f"✗ Error loading {csv_path}: {e}")

print(f"\nTotal variables loaded: {len(variables)}")
print(f"Expected: 426 observations per variable (1990-01 to 2025-06)")

## 4. Augmented Dickey-Fuller Test Implementation

In [ ]:
def perform_adf_test(timeseries, name):
    """
    Perform Augmented Dickey-Fuller test on a time series.
    
    Parameters:
    -----------
    timeseries : array-like
        The time series data to test
    name : str
        Name of the variable for reporting
    
    Returns:
    --------
    dict : Dictionary containing ADF test results
    """
    # Remove NaN values
    timeseries = timeseries[~np.isnan(timeseries)]
    
    # Perform ADF test
    result = adfuller(timeseries, autolag='AIC')
    
    # Extract results
    adf_result = {
        'Variable': name,
        'ADF Statistic': result[0],
        'P-Value': result[1],
        'Lags Used': result[2],
        'Observations': result[3],
        'Critical Value (1%)': result[4]['1%'],
        'Critical Value (5%)': result[4]['5%'],
        'Critical Value (10%)': result[4]['10%'],
        'Stationary (α=0.05)': 'Yes' if result[1] < 0.05 else 'No'
    }
    
    return adf_result

# Test the function on one variable
print("Example ADF Test:")
sample_result = perform_adf_test(variables['beefprice']['data'], 'beefprice')
for key, value in sample_result.items():
    print(f"{key}: {value}")

## 5. Run ADF Tests on All Variables

In [ ]:
# Run ADF tests on all variables
adf_results = []

for var_name, var_data in variables.items():
    result = perform_adf_test(var_data['data'], var_name)
    adf_results.append(result)
    print(f"✓ ADF test completed for {var_name}")

# Create a DataFrame for easy viewing
adf_df = pd.DataFrame(adf_results)
print("\n" + "="*100)
print("ADF TEST RESULTS SUMMARY")
print("="*100)

## 6. Display Stationarity Results

In [ ]:
# Display comprehensive results table
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print(adf_df.to_string(index=False))
print("\n" + "="*100)

# Summary statistics
stationary_vars = adf_df[adf_df['Stationary (α=0.05)'] == 'Yes']
non_stationary_vars = adf_df[adf_df['Stationary (α=0.05)'] == 'No']

print(f"\nSTATIONARITY SUMMARY (Significance Level α=0.05):")
print(f"Stationary Variables ({len(stationary_vars)}): {', '.join(stationary_vars['Variable'].tolist())}")
print(f"Non-Stationary Variables ({len(non_stationary_vars)}): {', '.join(non_stationary_vars['Variable'].tolist())}")
print(f"\nTotal Variables Tested: {len(adf_df)}")
print(f"Percentage Stationary: {(len(stationary_vars)/len(adf_df)*100):.1f}%")

## 7. Fractional Differencing (Lopez de Prado, 2018)

Apply fixed-width window fractional differencing only to non-stationary series. Stationary series are kept in levels.

In [ ]:
def compute_ffd_weights(d, tau=1e-5):
    """
    Compute weights for fixed-width window fractional differencing.
    Uses the binomial series expansion: w_k = -w_{k-1} * (d - k + 1) / k
    """
    weights = [1.0]
    for k in range(1, 10000):
        w = -weights[-1] * (d - k + 1) / k
        if abs(w) < tau:
            break
        weights.append(w)
    return np.array(weights)

def apply_ffd_to_series(series, d, weights):
    """
    Apply fractional differencing to a series using precomputed weights.
    First (width-1) values will be NaN.
    """
    if np.all(np.isnan(series)):
        return series
    
    width = len(weights)
    ffd = np.full_like(series, np.nan)
    
    for i in range(width - 1, len(series)):
        window = series[i - width + 1:i + 1]
        ffd[i] = np.dot(weights, window)
    
    return ffd

In [ ]:
def fracdiff_with_existing_adf(variables, adf_df, alpha=0.05, d_grid=None, 
                               tau=1e-5, log_transform=True, columns=None):
    """
    Create aligned DataFrame with optimal FFD of non-stationary series.
    Tests on the RAW log-transformed series for accuracy.
    """
    if d_grid is None:
        d_grid = np.linspace(0, 1, 201)  # Finer grid for better accuracy
    
    if columns is None:
        columns = list(variables.keys())
    
    # Build initial DataFrame with all dates
    all_dates = set()
    for var_name in columns:
        if var_name in variables:
            all_dates.update(variables[var_name]['dates'])
    
    all_dates = sorted(all_dates)
    df_aligned = pd.DataFrame({'date': all_dates})
    df_aligned.set_index('date', inplace=True)
    
    # Add each variable and find optimal d
    summary_rows = []
    
    for var_name in columns:
        if var_name not in variables:
            continue
        
        var_data = variables[var_name]
        dates = var_data['dates']
        series = var_data['data'].astype(float)
        
        # Apply log transform if requested
        if log_transform:
            series = np.log(series + 1e-8)  # Avoid log(0)
        
        # Create temp series on original dates
        temp_series = pd.Series(series, index=dates)
        
        # Align to full date range (forward fill)
        aligned_series_raw = temp_series.reindex(df_aligned.index)
        aligned_series_raw = aligned_series_raw.fillna(method='ffill')
        aligned_series_np = aligned_series_raw.values
        
        # Determine if original raw series is stationary
        series_clean = series[~np.isnan(series)]
        adf_result_orig = adfuller(series_clean, autolag='AIC', regression='c')
        is_stationary_orig = adf_result_orig[1] < alpha
        
        # Find optimal d if not stationary
        # TEST ON THE RAW (UNALIGNED) SERIES first for accurate d determination
        if is_stationary_orig:
            optimal_d = 0
            adf_pval_final = adf_result_orig[1]
            lags_used = adf_result_orig[2]
            notes = "Already stationary (d=0)"
            # For stationary series, use the aligned raw data
            final_series_np = aligned_series_np.copy()
        else:
            # Find optimal d by testing on RAW (not forward-filled) series
            optimal_d = None
            for d in d_grid:
                weights = compute_ffd_weights(d, tau=tau)
                ffd_raw = apply_ffd_to_series(series, d, weights)  # Apply to raw log-transformed series
                ffd_clean = ffd_raw[~np.isnan(ffd_raw)]
                
                if len(ffd_clean) < 3:
                    continue
                
                result = adfuller(ffd_clean, autolag='AIC', regression='c')
                if result[1] < alpha:
                    optimal_d = d
                    adf_pval_final = result[1]
                    lags_used = result[2]
                    break  # Return first d that passes (minimum)
            
            # If no d found in initial grid, use extended search up to 1.0
            if optimal_d is None:
                extended_grid = np.linspace(d_grid[-1], 1.0, 100)
                for d in extended_grid[1:]:  # Skip the first point (already tested)
                    weights = compute_ffd_weights(d, tau=tau)
                    ffd_raw = apply_ffd_to_series(series, d, weights)  # Test on raw
                    ffd_clean = ffd_raw[~np.isnan(ffd_raw)]
                    
                    if len(ffd_clean) < 3:
                        continue
                    
                    result = adfuller(ffd_clean, autolag='AIC', regression='c')
                    if result[1] < alpha:
                        optimal_d = d
                        adf_pval_final = result[1]
                        lags_used = result[2]
                        break
            
            if optimal_d is None:
                # If still no d found, use max d in grid
                optimal_d = d_grid[-1]
                notes = f"No d in grid passed ADF; used max d={optimal_d:.3f}"
            else:
                notes = f"Found optimal d={optimal_d:.3f}"
            
            # Now apply the FFD to ALIGNED series for storage in DataFrame
            weights = compute_ffd_weights(optimal_d, tau=tau)
            final_series_np = apply_ffd_to_series(aligned_series_np, optimal_d, weights)
        
        # Add final series to aligned DataFrame
        df_aligned[var_name] = final_series_np
        
        # Record summary
        summary_rows.append({
            'Variable': var_name,
            'Original Stationary': 'Yes' if is_stationary_orig else 'No',
            'Optimal d': optimal_d if optimal_d is not None else np.nan,
            'ADF p-value (final)': adf_pval_final,
            'Lags Used': lags_used,
            'Notes': notes
        })
    
    summary_df = pd.DataFrame(summary_rows)
    
    return df_aligned, summary_df

In [ ]:
# Run the fractional differencing pipeline
print("="*100)
print("FRACTIONAL DIFFERENCING PIPELINE")
print("="*100)
d_grid = np.linspace(0, 1, 201)  # 201 points for finer granularity
features_df, summary_df = fracdiff_with_existing_adf(
    variables=variables,
    adf_df=adf_df,
    alpha=0.05,
    d_grid=d_grid,
    tau=0.01,  # Optimized: 12-month window, minimal data loss
    log_transform=True,
    columns=list(variables.keys())
)

print("\n" + "="*100)
print("FRACTIONAL DIFFERENCING SUMMARY")
print("="*100)
print(summary_df.to_string(index=False))

print("\n" + "="*100)
print("FINAL FEATURES DATAFRAME")
print("="*100)
print(f"Shape: {features_df.shape}")
print(f"Date range: {features_df.index.min()} to {features_df.index.max()}")
print("\nFirst few rows:")
print(features_df.head())
print("\nData types:")
print(features_df.dtypes)
print("\nMissing values:")
print(features_df.isnull().sum())

## 8. Complete Workflow Summary

The notebook implements a full feature engineering pipeline:
1. **Load & Filter**: Extract data from Jan 1990 to Jun 2025, 1 observation per month-year
2. **Stationarity Test**: ADF test to identify non-stationary series
3. **Fractional Differencing**: Apply Lopez de Prado's FFD only to non-stationary series
4. **Alignment & Output**: Create a unified modeling DataFrame with all series properly transformed

## 9. Data Visualization and Inspection

In [ ]:
# Display detailed data inspection
print("="*100)
print("DETAILED DATA INSPECTION")
print("="*100)

# Get the last 10 non-NaN rows for each column
print("\n1. LAST 10 ROWS WITH DATA (showing end of time series):")
print("="*100)
print(features_df.tail(10))

print("\n\n2. SUMMARY STATISTICS:")
print("="*100)
print(features_df.describe())

print("\n\n3. DATA AVAILABILITY PER VARIABLE:")
print("="*100)
for col in features_df.columns:
    valid_count = features_df[col].notna().sum()
    pct_valid = (valid_count / len(features_df)) * 100
    print(f"{col:20s}: {valid_count:4d} / {len(features_df)} rows ({pct_valid:5.1f}% available)")

print("\n\n4. SAMPLE OF COMPLETE DATA (rows with all values present):")
print("="*100)
complete_rows = features_df.dropna()
if len(complete_rows) > 0:
    print(f"Found {len(complete_rows)} complete rows (no missing values)")
    print("\nLast 5 complete rows:")
    print(complete_rows.tail(5))
else:
    print("No rows with all values present (expected due to staggered data start dates)")
    print("\nShowing rows with most data available:")
    available_count = features_df.notna().sum(axis=1)
    top_rows_idx = available_count.nlargest(5).index
    print(features_df.loc[top_rows_idx])

print("\n\n5. CORRELATION MATRIX (non-stationary series after FD + stationary in levels):")
print("="*100)
# Drop rows with too many NaNs for correlation
corr_df = features_df.dropna()
if len(corr_df) > 0:
    correlation_matrix = corr_df.corr()
    print(correlation_matrix)
    
    # Find highest correlations (excluding diagonal)
    print("\n\nTOP CORRELATIONS (excluding self-correlation):")
    print("="*100)
    corr_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            col1 = correlation_matrix.columns[i]
            col2 = correlation_matrix.columns[j]
            corr_val = correlation_matrix.iloc[i, j]
            corr_pairs.append((col1, col2, corr_val))
    
    corr_pairs.sort(key=lambda x: abs(x[2]), reverse=True)
    for col1, col2, corr_val in corr_pairs[:10]:
        print(f"{col1:20s} <-> {col2:20s}: {corr_val:7.3f}")
else:
    print("Not enough complete data for correlation analysis")

print("\n\n6. TRANSFORMATION SUMMARY:")
print("="*100)
for idx, row in summary_df.iterrows():
    var = row['Variable']
    d_val = row['Optimal d']
    stationary = row['Original Stationary']
    pval = row['ADF p-value (final)']
    
    if d_val == 0:
        transform_type = "Level (no transformation)"
    else:
        transform_type = f"Fractionally Differenced (d={d_val:.3f})"
    
    print(f"{var:20s}: {transform_type:40s} | P-value: {pval:.6f}")

print("\n" + "="*100)
print("READY FOR MODELING!")
print("="*100)
print(f"\nYour features_df DataFrame has:")
print(f"  - {features_df.shape[0]} time periods")
print(f"  - {features_df.shape[1]} variables")
print(f"  - All series are now stationary and ready for time series modeling")
print(f"\nUse features_df.dropna() to get a dataset with complete observations.")
print(f"After dropna(): {len(features_df.dropna())} complete rows available for modeling.")

In [ ]:
# Create time series plots
fig, axes = plt.subplots(4, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, col in enumerate(features_df.columns):
    axes[idx].plot(features_df.index, features_df[col], linewidth=1.5, alpha=0.8)
    axes[idx].set_title(f'{col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Date')
    axes[idx].set_ylabel('Value')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('features_timeseries.png', dpi=150, bbox_inches='tight')
plt.show()

print("Time series plot saved as 'features_timeseries.png'")

In [ ]:
# Show a clean sample of the data
print("\n" + "="*100)
print("CLEAN DATA SAMPLE (Last 10 rows with complete data)")
print("="*100)

# Get rows with complete data (no NaN)
complete_data = features_df.dropna()
if len(complete_data) > 0:
    print(f"\nTotal rows with NO missing values: {len(complete_data)}")
    print("\nLast 10 rows:")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
        print(complete_data.tail(10).to_string())
else:
    print("No rows with complete data. Showing most recent available data:")
    print(features_df.tail(10).to_string())

print("\n" + "="*100)
print("DATA READY FOR EXPORT/MODELING")
print("="*100)
print(f"\nYou can now use features_df for your time series modeling!")
print(f"\nQuick usage examples:")
print(f"  1. Get clean data: clean_df = features_df.dropna()")
print(f"  2. Export to CSV: features_df.to_csv('features.csv')")
print(f"  3. Access specific column: features_df['beefprice']")
print(f"  4. Get date range: print(features_df.index.min(), 'to', features_df.index.max())")

## 10. Stationarity Verification

Verify that ALL series are now stationary after transformations.

In [ ]:
print("="*100)
print("STATIONARITY VERIFICATION - ALL TRANSFORMED SERIES")
print("="*100)

print("\n📊 ADF TEST RESULTS ON FINAL FEATURES (after transformation):")
print("="*100)

verification_results = []
for col in features_df.columns:
    series_clean = features_df[col].dropna().values
    if len(series_clean) >= 3:
        result = adfuller(series_clean, autolag='AIC', regression='c')
        adf_stat = result[0]
        pval = result[1]
        critical_5pct = result[4]['5%']
        
        is_stationary = "✅ YES" if pval < 0.05 else "❌ NO"
        
        verification_results.append({
            'Variable': col,
            'ADF Statistic': adf_stat,
            'P-Value': pval,
            'Critical (5%)': critical_5pct,
            'Stationary?': is_stationary
        })

verification_df = pd.DataFrame(verification_results)
print(verification_df.to_string(index=False))

print("\n" + "="*100)
print("SUMMARY")
print("="*100)

stationary_count = sum(1 for row in verification_results if row['P-Value'] < 0.05)
total_count = len(verification_results)

print(f"\n✅ Stationary Series: {stationary_count} / {total_count}")
print(f"❌ Non-Stationary Series: {total_count - stationary_count} / {total_count}")

if stationary_count == total_count:
    print(f"\n🎉 SUCCESS! All {total_count} variables are now STATIONARY and ready for modeling!")
else:
    print(f"\n⚠️  Warning: {total_count - stationary_count} series still non-stationary")